In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
import itertools

In [3]:
input_file_path = r'C:\Users\11921\Downloads\data\train_review.json'

sc = SparkContext.getOrCreate()
bid_uid_pair = sc.textFile(input_file_path).map(lambda line:json.loads(line))\
                        .map(lambda jsonLine:(jsonLine['business_id'],jsonLine['user_id']))\
                        .groupByKey().mapValues(lambda userList: list(set(userList))).collect()

In [10]:
bid_uid_dict = dict()
for pair in bid_uid_pair:
    bid = pair[0]
    uid_list = pair[1]
    bid_uid_dict[bid] = uid_list
len(bid_uid_dict)

10253

In [13]:
bid_list = list(bid_uid_dict.keys())
bid_candidate_tuple_list = list(itertools.combinations(bid_list, 2))
len(bid_candidate_tuple_list)

52556878

In [ ]:
ground_truth_list = list()
for candidate_pair in bid_candidate_tuple_list:
    can1 = candidate_pair[0]
    can2 = candidate_pair[1]
    ul1 = bid_uid_dict[can1]
    ul2 = bid_uid_dict[can2]
    jaccard = len(set(ul1).intersection(set(ul2)))/len(set(ul1).union(set(ul2)))
    if jaccard >= 0.05:
        ground_truth_list.append((can1, can2, jaccard))

In [19]:
fileWritter = open('task1GroudTruth.txt','w')
i = 0
for threeTuple in ground_truth_list:
    if i!=len(ground_truth_list)-1:
        fileWritter.write(str(threeTuple)+'\n')
    else:
        fileWritter.write(str(threeTuple)+'\n')
    i+=1
fileWritter.close()